[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/AGENslab/Python_learning-Sesion_1/blob/main/notebooks/example_solutions.ipynb
)

# 🧪 Sesión 1 – Fundamentos de Python para Ciencias Biológicas

**22/08/2025 – Dra. Carol Moraga e Ing. Felipe Gómez**

Bienvenid@. En este notebook practicaremos:
- Sintaxis básica de Python
- Estructuras de datos
- Control de flujo y funciones
- Lectura de un FASTA, GC% y k-mers

## 📁 Configuración mínima

Si corres en **Colab**, ejecuta la celda de abajo para subir el archivo `data/ejemplo.fasta` desde tu equipo (o monta Drive).

In [21]:
# En Colab, descomenta para subir archivos manualmente
#!wget https://raw.githubusercontent.com/AGENslab/Python_learning-Sesion_1/refs/heads/main/data/ejemplo.fasta

## 🔤 Sintaxis básica: variables y tipos

In [ ]:
entero = 42
pi = 3.14159
mensaje = "Hola bio-mundo"
es_valido = True

print(type(entero), type(pi), type(mensaje), type(es_valido))
assert isinstance(entero, int)
assert isinstance(mensaje, str)

## 🧱 Estructuras de datos: listas y diccionarios

In [ ]:
# Lista de longitudes de secuencias (ficticias)
longitudes = [120, 98, 301, 250]
# Diccionario: id -> especie
metadatos = {"seq1": "E. coli", "seq2": "A. thaliana"}

# Operaciones básicas
longitudes.append(180)
promedio = sum(longitudes) / len(longitudes)

print("N:", len(longitudes), "Promedio:", round(promedio, 2))
print("Especies:", list(metadatos.values()))

assert len(longitudes) == 5
assert "seq1" in metadatos

## 🔁 Control de flujo: `if`, `for`

In [ ]:
# Clasifica longitudes en cortas/medias/largas
clasificacion = []
for L in longitudes:
    if L < 120:
        clasificacion.append("corta")
    elif L <= 250:
        clasificacion.append("media")
    else:
        clasificacion.append("larga")

clasificacion

## 🧩 Funciones

In [25]:
def es_base(b):
    return b in {"A", "C", "G", "T"}

# Pruebas rápidas
assert es_base("A") and not es_base("N")

## 📄 Lectura de FASTA (sin librerías externas)

Implementaremos una función mínima que lea un archivo FASTA simple y retorne un diccionario `id -> secuencia`.

In [ ]:
from pathlib import Path

def leer_fasta(path):
    # Lee un FASTA simple y retorna dict id->secuencia (en mayúsculas, sin espacios).
    seqs = {}
    current_id = None
    parts = []
    with open(path, "r") as fh:
        for line in fh:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if current_id is not None:
                    seqs[current_id] = "".join(parts).upper()
                current_id = line[1:].split()[0]
                parts = []
            else:
                parts.append(line)
        if current_id is not None:
            seqs[current_id] = "".join(parts).upper()
    return seqs

# Carga ejemplo (ajusta ruta si es necesario)
fasta_path = Path("/content/ejemplo.fasta")
if not fasta_path.exists():
    print("No se encontró /content/ejemplo.fasta Sube el archivo o ajusta la ruta.")
else:
    seqs = leer_fasta(fasta_path)
    print("Cargadas:", len(seqs), "secuencias")
    # Muestra 1 ejemplo
    for k, v in list(seqs.items())[:1]:
        print(k, v[:60] + ("..." if len(v) > 60 else ""))

## 🧮 GC% por secuencia y global

Escribe una función `gc_percent(seq)` que retorne el porcentaje de G y C sobre A/C/G/T (ignora otras letras).

In [27]:
def gc_percent(seq):
    bases = [b for b in seq if b in "ACGT"]
    if not bases:
        return 0.0
    g = seq.count("G")
    c = seq.count("C")
    return 100.0 * (g + c) / len(bases)

# Pruebas
assert abs(gc_percent("GCGC") - 100.0) < 1e-6
assert abs(gc_percent("ATAT") - 0.0) < 1e-6

In [ ]:
# Si cargaste 'seqs', calcula GC% por id y el global
if 'seqs' in globals():
    gc_por_id = {sid: gc_percent(s) for sid, s in seqs.items()}
    gc_global = sum(gc_por_id.values()) / len(gc_por_id) if gc_por_id else 0.0
    print("GC% global:", round(gc_global, 2))
    list(gc_por_id.items())[:5]

## 🔢 Conteo de k-mers

Implementa `contar_kmers(seq, k)` que retorne un diccionario con la frecuencia de cada k-mer (sólo A/C/G/T). Luego crea un *ranking* de los más frecuentes.

In [29]:
from collections import Counter

def contar_kmers(seq, k=3):
    seq = "".join([b for b in seq if b in "ACGT"])
    counts = Counter()
    for i in range(len(seq) - k + 1):
        kmer = seq[i:i+k]
        counts[kmer] += 1
    return dict(counts)

# Pruebas
d = contar_kmers("AAACAAA", k=3)  # AAA aparece 2 veces: AAA, AAA (pos 0,4)
assert d.get("AAA", 0) == 2

In [ ]:
# Ranking de k-mers por secuencia y top global
def top_kmers_de_seqs(seqs, k=3, top=10):
    total = Counter()
    por_id = {}
    for sid, s in seqs.items():
        c = Counter(contar_kmers(s, k))
        por_id[sid] = c.most_common(top)
        total.update(c)
    return por_id, total.most_common(top)

if 'seqs' in globals():
    por_id, top_global = top_kmers_de_seqs(seqs, k=3, top=10)
    print("Top global (k=3):", top_global[:5])
    # Muestra top 5 del primer id
    first = next(iter(seqs)) if seqs else None
    if first:
        print("Top 5 de", first, por_id[first][:5])

## ✅ Cierre
- Practicaste variables, estructuras y funciones en Python.
- Leíste un FASTA simple y calculaste **GC%**.
- Implementaste un contador de **k-mers** con ranking.

**Desafíos opcionales:**
1. Generaliza `gc_percent` para devolver también el conteo de cada base.
2. Escribe una versión de `contar_kmers` que acepte alfabetos ambiguos (ej. `N`).
3. Grafica el top-10 de k-mers con `matplotlib`.